In [11]:
from dotenv import load_dotenv
import os

load_dotenv(verbose=True)
key = os.getenv('OPENAI_API_KEY')

In [12]:
from langchain.agents import tool
from typing import List, Dict, Annotated

In [13]:
from langchain_teddynote.tools import GoogleNews
from langchain_experimental.utilities import PythonREPL

In [14]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.agents import create_tool_calling_agent
from langchain.agents import AgentExecutor

In [15]:
from langchain_teddynote.messages import AgentStreamParser
from langchain_teddynote.messages import AgentCallbacks
from bs4 import BeautifulSoup
import requests
import re

### AentStreamParser, AgentCallbacks

- tool_callback: 도구 호출 출력을 처리하는 함수
- observation_callback: 관찰(Observation) 출력을 처리하는 함수
- result_callback: 최종 답변 출력을 처리하는 함수

### 도구 정의

In [16]:
@tool
def search_keyword(query: str) -> List[Dict[str, str]]:
    """Look up news by keyword"""

    print(f'검색어: {query}')

    news_tool = GoogleNews()

    return news_tool.search_by_keyword(query, k=2)

In [17]:
@tool
def python_repl_tool(
    code: Annotated[str, "The python code to execute to generate your chart."],
):
    """Use this to execute python code. If you want to see the output of a value,
    you should print it out with `print(...)`. This is visible to the user."""
    result = ""
    try:
        result = PythonREPL().run(code)
    except BaseException as e:
        print(f"Failed to execute. Error: {repr(e)}")
    finally:
        return result

### Agent 프롬프트 작성

In [18]:
# 프롬프트는 에이전트에게 모델이 수행할 작업을 설명하는 텍스트를 제공합니다. (도구의 이름과 역할을 입력)
prompt = ChatPromptTemplate.from_messages(
    [
        (                                           
            "system",
            "You are a helpful assistant. "             
            "Make sure to use the `search_news` tool for searching keyword related news."
            "Be sure to use the 'python_repl_tool' tool when make python code."
            "Be sure to use the 'naver_news_crawl' tool when naver news crawlling."
        ),
        ("placeholder", "{chat_history}"),          # 이전 대화 내용을 넣을 곳을 잡아둔다.
        ("human", "{input}"),                       # 사용자 입력

        # 에이전트가 검색하는 과정들이나 내용 등을 끄적이는 메모장 같은 공간을 플레이스 홀더로 만들어준다
        ("placeholder", "{agent_scratchpad}"),       
    ]
)

In [19]:
tools = [search_keyword, python_repl_tool]

### Agent 생성

In [20]:
llm = ChatOpenAI(
    api_key=key, 
    model='gpt-4o-mini', 
    temperature=0
)

agent = create_tool_calling_agent(llm, tools, prompt)

### AgentExecutor

In [21]:
agent_executor = AgentExecutor(
    agent=agent,                    # 각 단계에서 계획을 생성하고 행동을 결정하는 agent
    tools=tools,                    # agent 가 사용할 수 있는 도구 목록
    verbose=False,
    max_iterations=10,              # 최대 10번 까지 반복
    max_execution_time=10,          # 실행되는데 소요되는 최대 시간
    handle_parsing_errors=True      
)

### 중간 단계를 출력하는 사용자 정의 함수 정의

In [22]:
def tool_callback(tool) -> None:
    print(f'===== 도구 호출 =====')
    print(f"사용된 도구 이름 : {tool.get('tool')}")
    print(f'===== 도구 호출 =====')

In [23]:
def obervation_callback(observation) -> None:
    print(f'===== 관찰 내용 =====')
    print(f"Observation : {observation.get('observation')[0]}")
    print(f'===== 관찰 내용 =====')

In [24]:
def result_callback(result: str) -> None:
    print(f'===== 최종 답변 =====')
    print(result)
    print(f'===== 최종 답변 =====')

In [25]:
agent_callbacks = AgentCallbacks(
    tool_callback=tool_callback, 
    observation_callback=obervation_callback, 
    result_callback=result_callback
)

### AgentStreamParser

In [26]:
agent_stream_parser = AgentStreamParser(agent_callbacks)

In [27]:
result = agent_executor.stream({'input': '랭체인에 대한 뉴스를 검색해주세요'})

for step in result:
    agent_stream_parser.process_agent_steps(step)

===== 도구 호출 =====
사용된 도구 이름 : search_keyword
===== 도구 호출 =====
검색어: 랭체인
===== 관찰 내용 =====
Observation : {'url': 'https://news.google.com/rss/articles/CBMibkFVX3lxTE1qMC1EYjRZOEpMNVBEVVpSNm1BX3A4a3FrdmEyOTdDNWdjMGR1ZEgzWlpHZnRFZ1pSUXdKa3JnOXBQZU5qU3dKS3Q5VUNwLVZxRFZmbG1SZzJVSUoyM2lJbnZRbzhYYUJJcnFHWmp3?oc=5', 'content': '랭체인 LangChain 이란 무엇인가? | 인사이트리포트 | 삼성SDS - Samsung SDS'}
===== 관찰 내용 =====
===== 최종 답변 =====
다음은 "랭체인"에 대한 최근 뉴스입니다:

1. [랭체인 LangChain 이란 무엇인가? | 인사이트리포트 | 삼성SDS](https://news.google.com/rss/articles/CBMibkFVX3lxTE1qMC1EYjRZOEpMNVBEVVpSNm1BX3A4a3FrdmEyOTdDNWdjMGR1ZEgzWlpHZnRFZ1pSUXdKa3JnOXBQZU5qU3dKS3Q5VUNwLVZxRFZmbG1SZzJVSUoyM2lJbnZRbzhYYUJJcnFHWmp3?oc=5)

2. [개발자를 위한 랭체인 기초 개념과 환경 설정법 - 요즘IT](https://news.google.com/rss/articles/CBMiWkFVX3lxTFBtcm1OMmpJOUxvdnlkd0M0dVFfaGhma3BrTDRaVEY2OXFwd2tfUTZHVkFNSkZJYkZLc0x2dzFWWDNldF9TaVhIU0NxSUxyMlpIRC1KNksyLWdNQQ?oc=5)

더 궁금한 점이 있으시면 말씀해 주세요!
===== 최종 답변 =====


In [28]:
result = agent_executor.stream({'input': 'AI 관련 뉴스를 검색해 주세요.'})

for step in result:
    # 중간 단계를 parser 를 사용하여 단계별로 출력
    agent_stream_parser.process_agent_steps(step)

===== 도구 호출 =====
사용된 도구 이름 : search_keyword
===== 도구 호출 =====
검색어: AI
===== 관찰 내용 =====
Observation : {'url': 'https://news.google.com/rss/articles/CBMiVkFVX3lxTFBXWmRIU2NLbkxQSk5kajJBZlE3cHBRY0dXd2NWazZtQmVndE1qYmNCTjlkUTZ0c1pVT0U0d1pneGJ3Rk1HZ3FieFdBX01nZ3h1ZzRScG1R?oc=5', 'content': '[AI연구소] AI로 작성한 이력서, 채용 담당자들은 알아 차릴까 - ZD넷 코리아'}
===== 관찰 내용 =====
===== 최종 답변 =====
다음은 최근 AI 관련 뉴스입니다:

1. [AI로 작성한 이력서, 채용 담당자들은 알아 차릴까 - ZD넷 코리아](https://news.google.com/rss/articles/CBMiVkFVX3lxTFBXWmRIU2NLbkxQSk5kajJBZlE3cHBRY0dXd2NWazZtQmVndE1qYmNCTjlkUTZ0c1pVT0U0d1pneGJ3Rk1HZ3FieFdBX01nZ3h1ZzRScG1R?oc=5)
   
2. [스태빌리티 AI, 사진을 3D로 변환하는 AI 모델 출시 - AI타임스](https://news.google.com/rss/articles/CBMiakFVX3lxTE1uVHhTbURoUmE0ZWgwb1dpUEl1X2V6RTl0My1JdFNUQ0EyTFMwOG5NaVFlOFFQS0RqeEVsSEhVYzJOcGZScENQT2czZTloelFhTldMZ256ck90azFwb1VRNzNBVmJqSUowTnc?oc=5)

더 많은 정보가 필요하시면 말씀해 주세요!
===== 최종 답변 =====


In [29]:
result = agent_executor.stream({'input': '10 + 20을 출력하는 파이썬 코드를 작성하고 실행해주세요'})

for step in result:
    # 중간 단계를 parser 를 사용하여 단계별로 출력
    agent_stream_parser.process_agent_steps(step)

Python REPL can execute arbitrary code. Use with caution.


===== 도구 호출 =====
사용된 도구 이름 : python_repl_tool
===== 도구 호출 =====
===== 관찰 내용 =====
Observation : 3
===== 관찰 내용 =====
===== 최종 답변 =====
10 + 20의 결과는 30입니다.
===== 최종 답변 =====
